<a href="https://colab.research.google.com/github/vpw/EIP2/blob/master/Session4/DNST_CIFAR10_AUG-BC-100-12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.initializers import VarianceScaling
from keras.regularizers import l2

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64 # or 128
num_classes = 10
epochs = 50 # increase to 250
depth = 100 # total num layers - 40 for	DN, 100 for DN BC
k = 12 # growth rate
compression = 0.5 # reduction
dropout_rate = 0.2 # None for augmented  # 0.2 for non augmented

learning_rate = 0.1 # initial
weight_decay = 1e-4 # DCNNN paper
momentum = 0.9 # Nesterov

num_blocks = 3 # number of dense blocks in the network


In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# convert to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize - TODO check channel mean and std normalization later
x_train /= 255
x_test /= 255
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]


170500096/170498071 [==============================] - 60s 0us/step


In [0]:

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [0]:
# Dense Block
def add_denseblock(input, num_layers, num_filter, growth_rate, dropout_rate = None):
    global compression
    global init_mrsa
    global weight_decay
    temp = input
    for _ in range(num_layers):
        BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay/2),
                                       beta_regularizer=l2(weight_decay/2))(temp)
        relu = Activation('relu')(BatchNorm)
        # 1x1 conv - bottleneck layer
        Conv2D_1_1 = Conv2D(int(4*growth_rate),
            (1,1),
            use_bias=False,
            padding='same',
            kernel_initializer=init_mrsa,
            kernel_regularizer=l2(weight_decay/2))(relu)
        # 3x3 conv
        Conv2D_3_3 = Conv2D(int(growth_rate),
            (3,3), 
            use_bias=False, 
            padding='same',
            kernel_initializer=init_mrsa,
            kernel_regularizer=l2(weight_decay/2))(Conv2D_1_1)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        # concat outputs
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        num_filter += growth_rate
        
        temp = concat
        
    return temp, num_filter


In [0]:
def add_transition(input, num_filter, dropout_rate = None):
    global compression
    global init_mrsa
    global weight_decay
    BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay/2), 
                                beta_regularizer=l2(weight_decay/2))(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), 
        kernel_initializer=init_mrsa,
        use_bias=False, 
        padding='same',
        kernel_regularizer=l2(weight_decay/2))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2), strides=(2,2))(Conv2D_BottleNeck)
    
    return avg, int(num_filter*compression)

In [0]:
def output_layer(input, size):
    global compression
    global weight_decay
    BatchNorm = BatchNormalization(gamma_regularizer=l2(weight_decay/2), 
                                beta_regularizer=l2(weight_decay/2))(input)
    relu = Activation('relu')(BatchNorm)
    #AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    # try global lavg pooling
    AvgPooling = AveragePooling2D(pool_size=(size,size))(relu)
    # try 2x2 with 2x2 stride - gives 4x4x27
    #AvgPooling = AveragePooling2D(pool_size=(2,2), strides=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output


In [23]:
# num layers in each block
num_layers = int((depth - 4)/num_blocks) 

input = Input(shape=(img_height, img_width, channel,))
init_mrsa = VarianceScaling(2.0) # mode = FAN_IN, distribution=normal, to get MRSA init
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/variance_scaling_initializer
# https://medium.com/intuitionmachine/notes-on-the-implementation-densenet-in-tensorflow-beeda9dd1504

# initial number of filters - output of 1st layer is 2 * growth rate
num_filter = k * 2 

output = Conv2D(num_filter,  
                (3,3), 
                kernel_initializer=init_mrsa,
                use_bias=False,
                padding='same',
                kernel_regularizer=l2(weight_decay/2))(input) # https://bbabenko.github.io/weight-decay/
#32x32, 24

print (f'num_layers {num_layers}')
for block in range(num_blocks): 
    # num_layers/2 due to bottleneck net - 1 1x1 and 1 3x3 in each step
    print (f'DN: block {block} DB : in num_filter {num_filter}')
    output, num_filter = add_denseblock(output, int(num_layers/2), num_filter, k, dropout_rate)
    print (f'DN: block {block} DB : out num_filter {num_filter}')
#32x32, 24+12=36, 16x16x18+12=30, 8x8x15+12=27
    if (block != (num_blocks-1)):
        print (f'DN: block {block} TR : in num_filter {num_filter}')
        output, num_filter = add_transition(output, num_filter, dropout_rate)
        print (f'DN: block {block} TR : out num_filter {num_filter}')
#32x32, 16x16x36/2=18, 8x8x30/2=15, 

#First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
#First_Transition = add_transition(First_Block, num_filter, dropout_rate)

#Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
#Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

#Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
#Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

#Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
#
# 8x8x27
output = output_layer(output, 8)


num_layers 32
DN: block 0 DB : in num_filter 24
DN: block 0 DB : out num_filter 216
DN: block 0 TR : in num_filter 216
DN: block 0 TR : out num_filter 108
DN: block 1 DB : in num_filter 108
DN: block 1 DB : out num_filter 300
DN: block 1 TR : in num_filter 300
DN: block 1 TR : out num_filter 150
DN: block 2 DB : in num_filter 150
DN: block 2 DB : out num_filter 342


In [24]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_200 (Conv2D)             (None, 32, 32, 24)   648         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_103 (BatchN (None, 32, 32, 24)   96          conv2d_200[0][0]                 
__________________________________________________________________________________________________
activation_103 (Activation)     (None, 32, 32, 24)   0           batch_normalization_103[0][0]    
__________________________________________________________________________________________________
conv2d_201

In [0]:
from keras.optimizers import SGD
sgd = SGD(lr=learning_rate,momentum=momentum, decay=0.0, nesterov=True)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
#              optimizer=Adam(),
              optimizer=sgd,
              metrics=['accuracy'])

In [32]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 355s 7ms/step - loss: 1.9812 - acc: 0.5309 - val_loss: 2.4337 - val_acc: 0.4832
Epoch 2/50
50000/50000 [==============================] - 331s 7ms/step - loss: 1.4791 - acc: 0.6947 - val_loss: 1.7511 - val_acc: 0.6340
Epoch 3/50
50000/50000 [==============================] - 331s 7ms/step - loss: 1.2615 - acc: 0.7559 - val_loss: 1.7131 - val_acc: 0.6457
Epoch 4/50
50000/50000 [==============================] - 331s 7ms/step - loss: 1.1185 - acc: 0.7915 - val_loss: 1.3804 - val_acc: 0.7301
Epoch 5/50
50000/50000 [==============================] - 331s 7ms/step - loss: 1.0193 - acc: 0.8147 - val_loss: 1.0259 - val_acc: 0.8057
Epoch 6/50
50000/50000 [==============================] - 331s 7ms/step - loss: 0.9367 - acc: 0.8317 - val_loss: 1.1921 - val_acc: 0.7488
Epoch 7/50
50000/50000 [==============================] - 331s 7ms/step - loss: 0.8715 - acc: 0.8444 - val_loss: 1.0856 - 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
filepath="/content/gdrive/My Drive/EIP2/Session4/1/DCNN_1st_best.h5"

In [3]:
with open('/content/gdrive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/foo.txt

Hello Google Drive!

In [4]:
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint(filepath, verbose=1, save_best_only=True)

cb_list = [mcp]

Using TensorFlow backend.


In [5]:
# try next 50 epochs with same base LR, and reduceLROnPlateau
from keras.callbacks import ReduceLROnPlateau

rlr = ReduceLROnPlateau(patience=6, verbose=1, min_lr=0.001)

cb_list.append(rlr)

epochs=50

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=cb_list)

NameError: ignored

In [0]:
# try next 50 with LR/10 
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=cb_list)

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 13s 1ms/step
Test loss: 0.7949684534072876
Test accuracy: 0.7609


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')